In [9]:
from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from dataloading.nvidia import NvidiaResizeAndCrop, Normalize, NvidiaDataset
from network import PilotNet
from trainer import Trainer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
root_path = Path("/home/romet/data/datasets/ut/nvidia-data")
valid_paths = [root_path / "2021-05-20-13-59-00_e2e_sulaoja_10_10"]
tr = transforms.Compose([NvidiaResizeAndCrop(), Normalize()])
validset = NvidiaDataset(valid_paths, transform=tr)

In [15]:
trainer = Trainer("")
model = trainer.load_model("models/1-pilotnet-base/1cam/best.pt")

In [16]:
loader = DataLoader(validset, batch_size=1, shuffle=False, num_workers=0)
dataiter = iter(loader)
data = dataiter.next()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inputs = data['image'].to(device)
print(inputs.shape)

torch.Size([1, 3, 66, 258])


In [18]:
ONNX_FILE_PATH = 'models/1-pilotnet-base/1cam/best2.onnx'
torch.onnx.export(model, inputs, ONNX_FILE_PATH)